In [79]:
class convertCVtoText:
    @staticmethod
    def startConversion(fileName):
        import PyPDF2
        pdfFileObj = open(fileName,'rb')     #'rb' for read binary mode
        pdfReader = PyPDF2.PdfFileReader(pdfFileObj)
        noOfPages=pdfReader.numPages
        text=""
        for pages in range(0,pdfReader.numPages):
            pageObj = pdfReader.getPage(pages)          #'9' is the page number
            text+=pageObj.extractText()
        return text

In [80]:
class regularExp:
    @staticmethod
    def normalizeText(text):
        norText=""
        import re
        norText+= re.sub(r'[^a-zA-Z ]',r' ',text)
        return norText
    @staticmethod
    def Stemmer(text):
        from nltk.stem import PorterStemmer
        import nltk
        words=nltk.word_tokenize(text)
        stemmer = PorterStemmer()
        singles = [stemmer.stem(word) for word in words]
        #print(singles)
        singles=list(set(singles))
        #print(singles)
        stemmedText=" ".join(singles)
        #print(stemmedText)
        return stemmedText
        
        

In [81]:
class NLTKHelper:
    @staticmethod
    def findDocumentMatrix(totalCVText,minFrequency):
        from sklearn.feature_extraction.text import CountVectorizer
        vectorizer=CountVectorizer(stop_words='english',min_df=minFrequency)
        documentMatrix=vectorizer.fit_transform(totalCVText)
        vocabulary=vectorizer.vocabulary_
        return documentMatrix,vocabulary     

In [82]:
class CV:
     def __init__(self,path):
            self.fileName=path
            self.text=convertCVtoText.startConversion(self.fileName)
            self.stemedText=regularExp.Stemmer(self.text)
            self.cleanText=regularExp.normalizeText(self.text)
            self.cleanTextNew=regularExp.normalizeText(self.stemedText)


In [83]:
class databaseHandler:
    def __init__(self,password="ST.xavier51",databaseName="cvRanking"):
        import pymysql
        try:
            self.database=pymysql.connect("localhost","root","ST.xavier51","cvRanking")
            self.cursor = self.database.cursor()
        except Exception as e:
            print("error info %s"%e)
    def entryTheCV(self,data_CV):
        try:
#             dataReturned=self.checkDuplication(data_CV[1])
#             if(len(dataReturned)==0):
                add_CV="""insert into recordentry
                    (checkRecord,recordname)
                    values(%s,%s)"""
    #             add_CV="""insert into recordentry
    #                 (idrecordEntry,checkRecord,recordname)
    #                 values(%s,%s,%s)"""
                #data_CV=("yes","ram")
                print(data_CV)
                self.cursor.execute(add_CV,data_CV)
                self.database.commit()
                print("CV name is entered in database")
        except Exception as e:
            print("in entryTheCV\t"+str(e))
    def checkDuplication(self,name_CV):
        try:
            check_CV="""select* from recordentry as re where re.recordname=%s"""
            self.cursor.execute(check_CV,name_CV)
            result = self.cursor.fetchall()
        except Exception as e:
            print("in process checkDuplication\t"+str(e))
        if(len(result)==0):  
            return False
        else:
            return True  
    def closeConnection(self):
        self.database.close()


In [84]:
class CVManager:
    def __init__(self):
        self.CVAdded=None
        self.database=databaseHandler()
        self.CVList=[]
        self.CVPervoiusFileName=[]
        self.CVFileName=[]
        self.fileNamesWithPath=[]
        self.CVTextColl=[]
        self.CVTextCollTest=[]
        self.documentMatrix=[]
        self.vocabulary={}
        self.documentMatrixTest=[]
        self.vocabularyTest={}
    def list_CVs(self,rootPath):
            import os,json
            try:
                with open("C:\\Users\\sagar\\Desktop\\CV coll\\CVNameList.json","r") as file:
                    self.CVPervoiusFileName=json.load(file)
                file.close()
            except (FileNotFoundError, IOError):
                print("CV has not been added yet.\n Adding the CV")
            
            for root, dirs, files in os.walk(rootPath):
                for name in files:
                    self.CVFileName.append(name)
                    with open("C:\\Users\\sagar\\Desktop\\CV coll\\CVNameList.json","w") as file:
                        json.dump(self.CVFileName,file)
                    file.close()
                    self.fileNamesWithPath.append(os.path.join(root, name))
    def collectCV(self):
        self.checkCVList()
        if(self.CVAdded):
            for (cvPath,cvName) in zip(self.fileNamesWithPath,self.CVFileName):
                try:
                    if(not(self.database.checkDuplication(cvName))):
                        newCV=CV(cvPath)
                        self.CVList.append(newCV)
                        self.database.entryTheCV(("yes",cvName))
                    else:
                        newCV=CV(cvPath)
                        self.CVList.append(newCV)
                        print("%s named CV is already in database"%cvName)
                except Exception as e:
                    print("in collection of CV \t"+str(e))
    def checkCVList(self):
        if(set(self.CVFileName)& set(self.CVPervoiusFileName)):
            #if common
            print("all cv are added")
            self.CVAdded=False
        else:
            print("adding CV")
            self.CVAdded=True
            
    def collectCVText(self):
        if(self.CVAdded):
            for cv in self.CVList:
                self.CVTextColl.append(cv.cleanText) 
                self.CVTextCollTest.append(cv.cleanTextNew) 
    def findDocumentMatrix(self,minFrequency):
        import json
        try:
            if(self.newCVAdded):
                self.collectCVText()
                print("here")
                self.documentMatrix,self.vocabulary=NLTKHelper.findDocumentMatrix(self.CVTextColl,minFrequency)
                self.documentMatrixTest,self.vocabularyTest=NLTKHelper.findDocumentMatrix(self.CVTextCollTest,minFrequency)
                with open("C:\\Users\\sagar\\Desktop\\CV coll\\wholeCVText.json","w") as file:
                    json.dump(self.CVTextColl,file)
                file.close()
                with open("C:\\Users\\sagar\\Desktop\\CV coll\\wholeCVTextTest.json","w") as file:
                    json.dump(self.CVTextCollTest,file)
                file.close()
#                 with open("C:\\Users\\sagar\\Desktop\\CV coll\\documentMatrix.txt","w") as file:
#                     json.dump(list(self.documentMatrix),file)
#                 file.close()
#                 with open("C:\\Users\\sagar\\Desktop\\CV coll\\vocabulary.txt","w") as file:
#                     json.dump(self.vocabulary,file)
#                 file.close()
                #print( self.vocabulary)
            else:
                with open("C:\\Users\\sagar\\Desktop\\CV coll\\wholeCVText.json","r") as file:
                    self.CVTextColl=json.load(file)
                file.close()
                with open("C:\\Users\\sagar\\Desktop\\CV coll\\wholeCVTextTest.json","r") as file:
                    self.CVTextCollTest=json.load(file)
                file.close()
                self.documentMatrix,self.vocabulary=NLTKHelper.findDocumentMatrix(self.CVTextColl,minFrequency)
                self.documentMatrixTest,self.vocabularyTest=NLTKHelper.findDocumentMatrix(self.CVTextCollTest,minFrequency)
#                 with open("C:\\Users\\sagar\\Desktop\\CV coll\\documentMatrix.txt","r") as file:
#                     json.loads(self.documentMatrix,file)
#                 file.close()
#                 with open("C:\\Users\\sagar\\Desktop\\CV coll\\vocabulary.txt","r") as file:
#                     json.loads(self.vocabulary,file)
#                 file.close()
        except Exception as e:
                print("in reading/writing of saved files \t"+str(e))
                
                
            
        
        

In [85]:
manager=CVManager()

In [86]:
manager.list_CVs("C:\\Users\\sagar\\Desktop\\CV coll\\kailesh\\creative director")

CV has not been added yet.
 Adding the CV


In [87]:
manager.collectCV()

adding CV
('yes', 'Agnes-Szelag.pdf')
CV name is entered in database
('yes', 'Alison-Haller.pdf')
CV name is entered in database
('yes', 'Brad-Blackburn.pdf')
CV name is entered in database
('yes', 'Bruce-Jones.pdf')
CV name is entered in database
('yes', 'Carolyn-Carmel.pdf')
CV name is entered in database
('yes', 'Christina-Le-Sesne.pdf')
CV name is entered in database
('yes', 'Christopher-Mackey.pdf')
CV name is entered in database
('yes', 'Cosmo-Nardozza.pdf')
CV name is entered in database
('yes', 'Craig-Thacker.pdf')
CV name is entered in database
('yes', 'Cristopher-Schafer.pdf')
CV name is entered in database
('yes', 'Daniel-Montgomery.pdf')
CV name is entered in database
('yes', 'David-Cordon.pdf')
CV name is entered in database
('yes', 'David-Hindelang.pdf')
CV name is entered in database
('yes', 'David-Hunter.pdf')
CV name is entered in database
('yes', 'David-Moshier.pdf')
CV name is entered in database
('yes', 'Denise-Sucik.pdf')
CV name is entered in database
('yes', 'Des

In [93]:
manager.findDocumentMatrix(20)


in reading/writing of saved files 	'CVManager' object has no attribute 'newCVAdded'


In [89]:
#manager.CVTextColl

In [90]:
manager.vocabulary

{}

In [91]:
manager.vocabularyTest

{}

In [92]:
# for(k,v), (k2,v2) in zip(manager.vocabulary.items(), manager.vocabularyTest.items()):
#     print(k,v)